# Assignment 3 Dashboard
**Alumn:** Alvaro Alexis Muñoz Reynoso

**Email:** alvaro.munoz7503@alumnos.udg.mx

## Modules

In [5]:
import panel as pn
import panel.widgets as pnw

pn.extension('plotly')

import pandas as pd
import numpy as np

from scipy.stats import levy_stable
from scipy.stats import wrapcauchy
from scipy.spatial import distance

import plotly.graph_objects as go

import math

## Class Vec2d

In [6]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
    and also provides a bunch of high level functions
    """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:            
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y
            
    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)
    
    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)
    
    # rotate vector
    def rotated(self, angle):        
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)


## Trajectory Functions

### Brownian Motion

In [13]:
# Function to get Brownian Motion trajectory, using random 90 degree rotation angle directions
def bm_traj(speed=3, n_steps=1000, s_pos=[0,0]):
    trajectory_bw = pd.DataFrame(columns=['x_pos','y_pos'])
    temp_df = pd.DataFrame([{'x_pos':s_pos[0], 'y_pos':s_pos[1]}])

    trajectory_bw = pd.concat([trajectory_bw, temp_df], ignore_index=True)

    velocity = Vec2d(speed, 0) # Starting position for velocity vector

    for i in range(n_steps):
        turn_angle = np.random.choice([0, np.pi/2, np.pi, 3*np.pi/2])
        
        velocity = velocity.rotated(turn_angle)
        
        temp_df = pd.DataFrame([{'x_pos':trajectory_bw.x_pos[i] + velocity.x, 'y_pos':trajectory_bw.y_pos[i] + velocity.y}])
        trajectory_bw = pd.concat([trajectory_bw, temp_df], ignore_index=True)

    return trajectory_bw

### Correlated Random Walk

In [14]:
# Function to get correlated random walk trajectory using Wrapped Cauchy Distribution for turning angles on each step/iteration
def crw_traj(speed=5, coefficient=0.4, n_steps=1000, s_pos=[0,0]):
    
    r = wrapcauchy.rvs(loc=0, c=coefficient, size=n_steps)

    velocity = Vec2d(speed, 0) # Velocity vector in starting position

    trajectory_crw = pd.DataFrame(columns=['x_pos', 'y_pos'])
    temp_df = pd.DataFrame([{'x_pos':s_pos[0],'y_pos':s_pos[1]}])

    trajectory_crw = pd.concat([trajectory_crw,temp_df], ignore_index=True) 

    for i in range(n_steps):
        turn_angle = r[i]
        
        velocity = velocity.rotated(turn_angle)
        
        temp_df = pd.DataFrame([{'x_pos':trajectory_crw.x_pos[i] + velocity.x, 'y_pos':trajectory_crw.y_pos[i] + velocity.y}])
        trajectory_crw = pd.concat([trajectory_crw,temp_df], ignore_index=True)
        
    return trajectory_crw

### Levy Flight

In [15]:

def levy_traj(alpha=1.5, beta=1, m=0, speed=10, s_pos=[0,0,0], n_steps=1000, time_per_step=0.0004):
    # Initialize the dataFrame to store x, y values to trace trajectory, then set initial position for Data Frame
    trajectory = pd.DataFrame(columns=['x_pos', 'y_pos', 'z_pos'])
    temp_df = pd.DataFrame([{'x_pos':s_pos[0], 'y_pos':s_pos[1], 'z_pos':s_pos[2]}]) 

    trajectory = pd.concat([trajectory,temp_df], ignore_index=True)

    # Initialize the velocity vector with initial position
    velocity = Vec2d(speed, 0)

    # Random values using Lévy algorithm
    r = levy_stable.rvs(alpha, beta, loc=m, size=n_steps) #angle rotation values

    # trajectory x and y values are defined by lévy distribution random values.
    # On each iteration the difference between each step is calculated multiplying the value returned by the velocity 
    # vector with the current value of the random lavue from lévy distribution.
    for i in range(n_steps):
        step_length = levy_stable.rvs(alpha,beta)
        
        # Rotation angles for the vector are also random values from lévy distribution
        velocity = velocity.rotated(r[i])
        
        length_traveled = velocity.get_length() * step_length * time_per_step # times values (z)
        
        # new position values are stored in the trajectory dataframe
        temp_df = pd.DataFrame([{
            'x_pos':trajectory.x_pos[i] + velocity.x * step_length, 
            'y_pos':trajectory.y_pos[i] + velocity.y * step_length, 
            'z_pos':trajectory.z_pos[i] + length_traveled
            }]
        )
        trajectory = pd.concat([trajectory, temp_df], ignore_index=True)
    return trajectory
    

## Metric Functions

### Path Length Metric

In [10]:
# Function to get the path length of a given dataframe trajectory
def pl_traj(df, s_pos=[0]):
    pl = pd.DataFrame(columns=['y_pos'])
    temp_df = pd.DataFrame([{'y_pos':s_pos[0]}])

    pl = pd.concat([pl, temp_df], ignore_index=True)

    for i in range(len(df)-1): # minus one because len counts the dataframe header
        temp_df = pd.DataFrame([{'y_pos':pl.y_pos[i] + distance.euclidean(df.iloc[i], df.iloc[i+1])}])
        pl = pd.concat([pl, temp_df], ignore_index=True)
        
    return pl


### Mean Squared Displacement Metric

In [11]:
def msd(df, s_pos=0):
    n = 1 # MSD Formula n initial value
    msd_df = pd.DataFrame(columns=['y_pos'])
    temp_df = pd.DataFrame([{'y_pos':s_pos}])
    msd_df = pd.concat([msd_df, temp_df], ignore_index=True)
    
    for n in range(1,len(df)-1):
        
        eucladean_distances = np.array([distance.euclidean(df.iloc[i-n],df.iloc[i]) for i in range(n, df.shape[0],1)])
        dist_squared = eucladean_distances**2
        msd = np.mean(dist_squared)
        
        
        temp_df = pd.DataFrame([{'y_pos':msd}])
        msd_df = pd.concat([msd_df, temp_df], ignore_index=True)
        
    return msd_df

### Turning-angle distribution metric

In [12]:
def turning_angles(trajectory):
    turning_angles = []
    for i in range(1, len(trajectory)-1):
        # Calculate the turning angle
        a = trajectory.iloc[i-1]
        b = trajectory.iloc[i]
        c = trajectory.iloc[i+1]
        ab = b - a
        bc = c - b
        cross_product = np.cross(ab, bc)
        orient = np.sign(cross_product)
        if orient == 0:
            orient = 1
            
        denominator = np.linalg.norm(ab) * np.linalg.norm(bc) + np.finfo(float).eps #to avoid zeros
        
        angle = np.arcsin(np.linalg.norm(cross_product) / denominator)
        
        new_angle = angle * orient
        turning_angles.append(round(new_angle, 2))
        
    return pd.DataFrame(turning_angles, columns=['turning_angle'])

## Widgets

In [71]:
#radio group for trajectories
rg_traj = pn.widgets.RadioButtonGroup(name='Trajectories', value='BM', options=['BM', 'CRW', 'Levy'], width=300)


@pn.depends(rg_traj)
def traj_options(rg_traj):
    step_slider = pn.widgets.IntSlider(value=1000, start=10, end=5000, step=100, name="Number of steps")
    if rg_traj == 'BM':
        return step_slider
    elif rg_traj == 'CRW':
        pass
    elif rg_traj == 'Levy':
        alpha_input = pn.widgets.FloatInput(name='Alpha', value=0.5, step=0.1, start=0.1, end=2.0, width=75)
        beta_input = pn.widgets.FloatInput(name='Beta', value=1, step=0.1, start=0.1, end=1.0, width=75)
        m_input = pn.widgets.IntInput(name='M/Loc', value=0, step=1, start=0, end=10, width=75)
        
        speed_slider = pn.widgets.IntSlider(value=5, start=1, end=100, step=1, name="Speed")
        
        x_input = pn.widgets.IntInput(name='X pos', value=0, step=1, start=-500, end=500, width=125)
        y_input = pn.widgets.IntInput(name='Y pos', value=0, step=1, start=-500, end=500, width=125)
        
        return pn.Column(step_slider, speed_slider, pn.Row(alpha_input, beta_input, m_input), pn.Row(x_input,y_input))
    
    return None
        
pn.Column(rg_traj, traj_options)


BokehModel(combine_events=True, render_bundle={'docs_json': {'43fd7de6-8fdd-416a-8a10-e9566ff9097e': {'version…

In [66]:
rg_traj.value

'Levy'